
[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\91993\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [31]:
import numpy as np
import pandas as pd

In [32]:
import os
for dirname, _, filenames in os.walk('./Data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./Data/gender_submission.csv
./Data/test.csv
./Data/train.csv


In [33]:
DATA_DIR = './Data/'
TRAIN_DATA = 'train.csv'
TEST_DATA = 'test.csv'

In [118]:
train_data = pd.read_csv(DATA_DIR + TRAIN_DATA)
test_data = pd.read_csv(DATA_DIR + TEST_DATA)
test_ids = test_data["PassengerId"]
train_data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [119]:
def clean(data):
    data = data.drop(["Ticket", "Name", "PassengerId"], axis=1)
    cols = ["SibSp", "Parch", "Fare", "Age"]
    data.Cabin.fillna("NA", inplace=True)
    data.Cabin = data.Cabin.apply(lambda x : x[0])
    for col in cols:
        fillin_data = data.sort_values('Cabin').groupby('Cabin')[col].median()
        temp_med = data[col].median()
        data[col] = np.where(data[col].isnull(), data.Cabin.apply(lambda x: fillin_data[x] if fillin_data[x] > temp_med else temp_med), data[col])
        data[col].fillna(fillin_data['A'], inplace=True)
    data.Embarked.fillna("U", inplace=True)
    return data

train_data = clean(train_data)
test_data = clean(test_data)

C:\Users\91993\AppData\Local\Temp\ipykernel_27152\1161778953.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.Cabin.fillna("NA", inplace=True)
C:\Users\91993\AppData\Local\Temp\ipykernel_27152\1161778953.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

In [120]:
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1.0,0.0,7.2500,N,S
1,1,1,female,38.0,1.0,0.0,71.2833,C,C
2,1,3,female,26.0,0.0,0.0,7.9250,N,S
3,1,1,female,35.0,1.0,0.0,53.1000,C,S
4,0,3,male,35.0,0.0,0.0,8.0500,N,S


In [121]:
from sklearn import preprocessing

In [122]:
le = preprocessing.LabelEncoder()
cols = ["Sex", "Embarked", "Cabin"]

for col in cols:
    train_data[col] = le.fit_transform(train_data[col])
    test_data[col] = le.fit_transform(test_data[col])
    print (le.classes_)

train_data.head() 

['female' 'male']
['C' 'Q' 'S']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'N']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,1,22.0,1.0,0.0,7.2500,7,2
1,1,1,0,38.0,1.0,0.0,71.2833,2,0
2,1,3,0,26.0,0.0,0.0,7.9250,7,2
3,1,1,0,35.0,1.0,0.0,53.1000,2,2
4,0,3,1,35.0,0.0,0.0,8.0500,7,2


In [123]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = train_data["Survived"]
x = train_data.drop(["Survived"], axis=1)

X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [124]:
clf = LogisticRegression(random_state=0, max_iter=10000).fit(X_train, y_train) 

In [125]:
predictions = clf.predict(X_val)
from sklearn.metrics import accuracy_score
accuracy_score(y_val, predictions)

0.8212290502793296

In [126]:
submission_predictions = clf.predict(test_data)

In [127]:
df = pd.DataFrame({
    "PassengerId": test_ids,
    "Survived": submission_predictions
})

In [128]:
df.to_csv('submission.csv', index=False)